In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [79]:
import random as ran
from IPython.display import clear_output
from src import spotify, dp, utils, DB

db = DB()

In [80]:
genres_cat = ['k-pop', 'pop', 'hip-hop', 'ballad', 'jazz']

genres_dict = dict({
    "k-pop": "3dbLT62Cvs46Ju7a8gpr36",
    "pop": "3gzIK2dGSVvPBiohsTwEWZ",
    "hip-hop": "2FWquqPNxte8iqZ3ATQG0p",
    "ballad": "3UPjb91Fwm7u2tAm92Bk0p",
    "jazz": "7tIOwGFRTM08JdAeKOELGp"
})

In [81]:
mail_box_seed_list = list()

for _ in range(0, 10):
    genre_cnt = ran.randrange(1, 4)
    genre_cat = list()
    
    while len(genre_cat) != genre_cnt:
        ran_genres_cat = genres_cat[
            ran.randrange(0, len(genres_cat))
        ]
        
        if ran_genres_cat not in genre_cat:
            genre_cat.append(ran_genres_cat)

    mail_box_seed_list.append(genre_cat)
    
mail_box_seed_list

[['k-pop'],
 ['k-pop', 'jazz'],
 ['hip-hop', 'jazz', 'pop'],
 ['pop'],
 ['ballad', 'pop'],
 ['hip-hop'],
 ['ballad', 'pop'],
 ['jazz', 'ballad'],
 ['hip-hop'],
 ['jazz', 'pop']]

In [82]:
# 1. Get Token
token = spotify.get_token()

mail_boxes_list = list()
sel_track_list = list()
sel_tracks = pd.DataFrame()

for box_seed in mail_box_seed_list:
    sel_tracks = pd.DataFrame()
    
    for seed_genre in box_seed:
        seed_id = genres_dict[seed_genre]
        
        sel_tracks = sel_tracks.append(spotify.direct_sel(token, seed_id), ignore_index=True)
    
    sel_track_list.append(sel_tracks)
    mail_box_id = str(db.regist_mail_box(sel_tracks).inserted_id)
    
    mail_boxes_list.append(mail_box_id)

print(mail_boxes_list)
sel_track_list[0]

['623c41cba96fb402f17e71ab', '623c41cba96fb402f17e71ac', '623c41cba96fb402f17e71ad', '623c41cba96fb402f17e71ae', '623c41cca96fb402f17e71af', '623c41cca96fb402f17e71b0', '623c41cca96fb402f17e71b1', '623c41cca96fb402f17e71b2', '623c41cca96fb402f17e71b3', '623c41cca96fb402f17e71b4']


,id,name,artists,artists_name
0,3dbLT62Cvs46Ju7a8gpr36,Savage,6YVMFz59CuY7ngCxTxjpxE,aespa


In [85]:
features_list = list()
genre_seed_list = list()
for sel_track in sel_track_list:
    genres = spotify.get_genres(token, sel_track)
    genre_seed_list.append(genres)
    
    features = spotify.get_features(token, sel_track)
    features_list.append(features)
    
    db.observe_seed_zone(features)
    
print(genre_seed_list)
features_list[0]

[      count
k-pop     1,       count
k-pop     1
jazz      1,       count
k-pop     1
jazz      1
pop       1,     count
pop     1,       count
k-pop     1
pop       1,       count
k-pop     1,       count
k-pop     1
pop       1,       count
jazz      1
k-pop     1,       count
k-pop     1,      count
jazz     1
pop      1]


,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3dbLT62Cvs46Ju7a8gpr36,0.727,0.879,9,-1.167,0.129,0.124,0.000012,0.244,0.671,146.959


In [86]:
reco_list = list()
reco_features_list = list()

for idx, sel_track in enumerate(sel_track_list):
    features = features_list[idx]
    genres = genre_seed_list[idx]
    
    recos = spotify.get_recommend(sel_track, features, genres, token)
    reco_features = spotify.get_features(token, recos)
    
    reco_list.append(recos)
    reco_features_list.append(reco_features)

In [87]:
reco_list[0].head()

,id,name,artists,artists_name
0,2mKl38xBfLznqODvyaNQPq,Fire Saturday,7qxo9RTWfEoFKN7XGtNV6V,SECRET NUMBER
1,0CnpSTdL9l5vQM4YnzXtyo,Panorama,5r1tUTxVSgvBHnoDuDODPH,IZ*ONE
2,4zCIxSnVWpGNghERX4uWZF,SMILEY(Feat. BIBI),49muoiIu4uea4PO8vueUNN,YENA
3,6poVmpGU3y3jj1Z9xbbbH4,"Weapon (With Newnion, FLOOR) (Prod.Czaer)",0LyfQWJT6nXafLPZqxe9Of,Various Artists
4,7eVu7FI02cTicLEgVtUvwF,ZOO,49VtaZvoqBgZHQxSqlCUyp,SMTOWN


In [88]:
reco_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,2mKl38xBfLznqODvyaNQPq,0.735,0.837,9,-1.186,0.0434,0.2000,0.000000,0.0551,0.618,125.926
1,0CnpSTdL9l5vQM4YnzXtyo,0.726,0.963,9,-1.111,0.0916,0.0429,0.001570,0.1280,0.769,126.062
2,4zCIxSnVWpGNghERX4uWZF,0.652,0.910,9,-3.444,0.1340,0.0963,0.000013,0.0930,0.820,155.967
3,6poVmpGU3y3jj1Z9xbbbH4,0.575,0.931,9,-1.628,0.0867,0.0162,0.000001,0.4170,0.521,168.024
4,7eVu7FI02cTicLEgVtUvwF,0.823,0.923,9,-2.875,0.1580,0.1270,0.000000,0.0571,0.629,122.922


In [89]:
norm_features_list = list()

for idx, features in enumerate(features_list):
    reco_features = reco_features_list[idx]
    norm_features = dp.make_norm(features, reco_features)
    
    norm_features_list.append(norm_features)
    
norm_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3dbLT62Cvs46Ju7a8gpr36,0.588752,0.795841,0.818182,0.865135,0.403816,0.243239,0.000299,0.293996,0.548031,0.566892
1,2mKl38xBfLznqODvyaNQPq,0.602812,0.716446,0.818182,0.862458,0.063593,0.392625,0.000000,0.050631,0.464567,0.332773
2,0CnpSTdL9l5vQM4YnzXtyo,0.586995,0.954631,0.818182,0.873027,0.255167,0.083828,0.040153,0.144550,0.702362,0.334287
3,4zCIxSnVWpGNghERX4uWZF,0.456942,0.854442,0.818182,0.544250,0.423688,0.188791,0.000320,0.099459,0.782677,0.667160
4,6poVmpGU3y3jj1Z9xbbbH4,0.321617,0.894140,0.818182,0.800169,0.235692,0.031346,0.000037,0.516877,0.311811,0.801367


In [90]:
kmeans_list = list()

for idx, norm_features in enumerate(norm_features_list):
    print("-- 라운드 {}".format(idx + 1))
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=3)
    clear_output(wait=True)
    
    kmeans_list.append(kmeans)

-- 라운드 10
1 round : ECV 53.94%
2 round : ECV 71.98%
3 round : ECV 74.36%
4 round : ECV 74.63%
5 round : ECV 74.66%
6 round : ECV 74.67%
7 round : ECV 74.68%
8 round : ECV 74.68%
9 round : ECV 74.68%
10 round : ECV 74.68%
11 round : ECV 74.68%
Clustering End.


In [91]:
filterings = list()
recos_list = list()

for idx, kmeans in enumerate(kmeans_list):
    sel_track = sel_track_list[idx]
    recos = reco_list[idx]
    
    _filtering_music_list = dp.music_filtering(sel_track, kmeans)
    filterings.append(_filtering_music_list)
    
    reco_musics = [_ in _filtering_music_list for _ in recos['id']]
    recos_list.append(recos[reco_musics])
    
recos_list[0].head()

,id,name,artists,artists_name
2,4zCIxSnVWpGNghERX4uWZF,SMILEY(Feat. BIBI),49muoiIu4uea4PO8vueUNN,YENA
3,6poVmpGU3y3jj1Z9xbbbH4,"Weapon (With Newnion, FLOOR) (Prod.Czaer)",0LyfQWJT6nXafLPZqxe9Of,Various Artists
12,4rOODw637hsmsq0uzT0DN3,Advice,13rF01aOogvnkuQXOlgTW8,TAEMIN
15,5bMHrigI1EAmZdiyRzKoQi,ICONIC,6YVMFz59CuY7ngCxTxjpxE,aespa
26,7EUIqHUc6qKk7iVdG1W7RE,Let It,0tzSBCPJZmHTdOA3ZV2mN3,CL


In [92]:
for idx,_mail in enumerate(recos_list):
    mail_box_id = mail_boxes_list[idx]
    
    res = db.regist_mail(mail_box_id, _mail)
    mail_id = res.inserted_id
    
    print("우체통({})에\n추천 플레이리스트 MuLetter({})가 등록되었습니다.".format(
        mail_box_id
        ,str(mail_id)))

우체통(623c41cba96fb402f17e71ab)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71ba)가 등록되었습니다.
우체통(623c41cba96fb402f17e71ac)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71bb)가 등록되었습니다.
우체통(623c41cba96fb402f17e71ad)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71bc)가 등록되었습니다.
우체통(623c41cba96fb402f17e71ae)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71bd)가 등록되었습니다.
우체통(623c41cca96fb402f17e71af)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71be)가 등록되었습니다.
우체통(623c41cca96fb402f17e71b0)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71bf)가 등록되었습니다.
우체통(623c41cca96fb402f17e71b1)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71c0)가 등록되었습니다.
우체통(623c41cca96fb402f17e71b2)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71c1)가 등록되었습니다.
우체통(623c41cca96fb402f17e71b3)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71c2)가 등록되었습니다.
우체통(623c41cca96fb402f17e71b4)에
추천 플레이리스트 MuLetter(623c421ca96fb402f17e71c3)가 등록되었습니다.
